# Shared Analysis Functions for Reflex MUC 2023

## Notebook Summary
- Provides reusable helper functions for cleaning, transforming, and exporting study data.
- Centralizes shared statistics logic (confidence intervals, whiskers, and descriptive summaries).
- Contains reusable plotting routines for condition-level and block-level comparisons.
- Includes utility functions for peak processing and derived metrics used across analysis notebooks.


## Imports and Global Configuration
This cell imports analysis libraries and defines shared constants such as paths, condition order, and percentiles.


In [2]:
# Shared imports for data wrangling, statistics, and plotting.

import pandas as pd
import glob
import os
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import seaborn as sns
from scipy.signal import find_peaks, find_peaks_cwt, savgol_filter
import plotly.graph_objects as go

# Timestamp format used in result files.
timeFormat = "%Y-%m-%dT%H:%M:%S.%fZ"

# Central output and input directories reused by multiple notebooks.
export_data = "../export/data/"
export_img = "../export/img/complete/"
export_img_single = "../export/img/single/"
export_img_questionnaire = "../export/img/questionnaire/"

export_data_spss = "../export/spss/"

path = "../data/"

path_questionnaires = "../questionnaires/"

# Canonical condition order for grouped analyses and plots.
condition_names = ['No Feedback', 'Tactile Feedback', 'Visual Feedback', 'Combined Feedback']

# Extra quantiles included in descriptive statistics exports.
percentiles = [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

sns.set_theme()


## Export Helpers for Data and Results
This cell defines functions that persist complete and experiment-only datasets, including validity-filtered result exports.


In [4]:
def saveData(data_complete):
    # Export the full merged tracking dataset.
    data_complete.to_csv(rf'../export/data/data_all.csv', sep=";", index=False)

    # Keep only experiment blocks (training blocks have negative block ids).
    data_experiment_only = data_complete[data_complete['Block'] >= 0]

    data_experiment_only.to_csv(rf'../export/data/data_experiment.csv', sep=";", index=False)

def saveResults(results_complete):
    # Export all task result rows.
    results_complete.to_csv(rf'../export/data/results_all.csv', sep=";", index=False)

    # Exclude participant 3 from validity-filtered exports.
    results_valid = results_complete[results_complete['ProbandId'] != 3]
    
    results_valid.to_csv(rf'../export/data/results_valid_all.csv', sep=";", index=False)

    # Keep only experiment blocks for both complete and validity-filtered datasets.
    results_experiment_only = results_complete[results_complete['BlockId'] >= 0]
    results_valid_experiment_only = results_valid[results_valid['BlockId'] >= 0]

    results_experiment_only.to_csv(rf'../export/data/results_experiment.csv', sep=";", index=False)
    results_valid_experiment_only.to_csv(rf'../export/data/results_experiment_valid.csv', sep=";", index=False)


## Peak Data Load and Save Utilities
This cell provides helper functions to deserialize and serialize array-like peak columns when reading from or writing to CSV.


In [5]:
def loadPeaks(recoverArrayColumns = []):
    result = pd.read_csv(rf'{export_data}all_peaks.csv', sep= ";")

    # Convert serialized list strings back into numpy arrays after CSV import.
    for col in recoverArrayColumns:
        result[col] = result[col].apply(lambda item: np.fromstring(item.replace('[','').replace(']',''), dtype=float, sep="|"))
   
    return result

def savePeaks(result_all, writeArrayColumns = []):
    # Serialize numpy arrays with a custom separator so they survive CSV export/import.
    for col in writeArrayColumns:
        result_all[col] = result_all[col].apply(lambda item: np.array2string(item, separator="|"))
    
    result_all.to_csv(rf'{export_data}all_peaks.csv', sep= ";")


## Last Layer-Change Lookup
This cell adds a helper that finds the most recent non-zero layer-change event before a given row index.


In [ ]:
def getLastLayerChange(index):
    # Find the latest non-zero layer change before the current row index.
    data = data_complete[(data_complete['LayerChange'] != 0) & (data_complete.index < index)].tail(1)

    date = data['Date'].values[0]
    idx = data.index.values[0]
    
    return (date, idx)


## Plot Margin Helper
This cell defines a plotting utility that expands axis limits by a configurable relative margin.


In [2]:
def add_margin(ax,x=0.05,y=0.05):
    # Expand axis limits by a relative margin to avoid clipped data and labels.
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    xmargin = (xlim[1]-xlim[0])*x
    ymargin = (ylim[1]-ylim[0])*y

    ax.set_xlim(xlim[0]-xmargin,xlim[1]+xmargin)
    ax.set_ylim(ylim[0]-ymargin,ylim[1]+ymargin)


## Statistical Utility Functions
This cell provides reusable helpers to compute 95% confidence intervals and Tukey-style whiskers.


In [ ]:
def computeCI(df, m= 'mean', c='count', s='std'):
    # Add 95% confidence interval bounds using a normal approximation.
    df['ci95_hi'] = df[m] + 1.96*df[s]/(df[c].apply(np.sqrt))
    df['ci95_lo'] = df[m] - 1.96*df[s]/(df[c].apply(np.sqrt))

def computeWhiskers(df, desc, col_names, q1='25%', q3='75%'):
    desc['iqr'] = desc[q3] - desc[q1]

    # Compute Tukey inner-fence bounds (1.5 * IQR).
    desc['lower_bound'] = desc[q1] - 1.5 * desc['iqr']
    desc['upper_bound'] = desc[q3] + 1.5 * desc['iqr']    

    lower_whiskers = []
    upper_whiskers = []

    for col_name in col_names:
        lower_bound = desc['lower_bound'].T[col_name]
        upper_bound = desc['upper_bound'].T[col_name]

        # Whiskers are the furthest observed values within the inner fences.
        l = df[df[col_name] >= lower_bound][col_name].min()
        u = df[df[col_name] <= upper_bound][col_name].max()

        lower_whiskers.append(l)
        upper_whiskers.append(u)

    desc['lower_whisker'] = lower_whiskers
    desc['upper_whisker'] = upper_whiskers


## Condition-Based Statistics and Plot Generation
This cell computes descriptive statistics and exports summary tables and plots across conditions, blocks, and trial progression.


In [ ]:
def computeStatistics(data_complete, columnName = 'DurationMS', filePrefix='duration', title='Duration', label_y='Time (s)'):
    conditions = data_complete.groupby(['Condition'])

    # Pivot to one column per condition, aligned by within-condition trial counter.
    # Reindex keeps the predefined condition order across exports and plots.
    condition_duration = data_complete.pivot_table(columns=['Condition'], values=[columnName], index=['countCondition'])[columnName].reindex(columns=condition_names)

    display(condition_duration)

    # Export condition-level trial values.
    condition_duration.to_csv(rf'{export_data}{filePrefix}_allTrials.csv', sep=";", index=False)

    # Average metric per block and condition for block-wise comparison.
    condition_duration_block = data_complete.groupby(['Condition', 'BlockId'])[columnName].mean().unstack(level=0).reindex(columns=condition_names)

    # Rearrange the same table so conditions become rows and blocks columns.
    condition_block_duration = condition_duration_block.stack().unstack(level = 0)

    display(condition_block_duration)

    condition_block_duration.to_csv(rf'{export_data}{filePrefix}_trialsBlockCondition.csv', sep=";", index=False)

    # Compute descriptive statistics and confidence intervals per condition and overall.
    durations_desc = condition_duration.describe(percentiles=percentiles)

    stats = pd.DataFrame(durations_desc).T
    stats_total = pd.DataFrame(data_complete[columnName].describe()).T

    computeCI(stats)
    computeCI(stats_total)

    computeWhiskers(condition_duration, stats, condition_names)
    # computeWhiskers(condition_duration, stats_total, [])

    display(durations_desc)

    stats_complete = pd.concat([stats, stats_total], ignore_index=False)

    display(stats_complete.T)

    stats_complete.to_csv(rf'{export_data}{filePrefix}_stats.csv', sep=";", index=True)

    # Pivot by trial id to inspect learning effects over the block sequence.
    durations = data_complete.pivot_table(index='TrialId', columns=['Condition'], values=columnName).reindex(columns=condition_names)

    display(durations)

    durations.to_csv(rf'{export_data}{filePrefix}_trialsBlock.csv', sep=";", index=False)

    # Line plot: metric per trial split by condition.
    fig0, ax0 = plt.subplots(figsize=(25,8))

    condition_duration.plot(ax = ax0)

    ax0.set_xlabel('Trial')
    ax0.set_ylabel(label_y)

    plt.title(f'{title} of each trial by Condition')
    

    fig0.savefig(rf'{export_img}{filePrefix}_allTrials.png')
    fig0.savefig(rf'{export_img}{filePrefix}_allTrials.svg')
    
    plt.show()

    # Line plot over trial number (block progression).
    fig1, ax1 = plt.subplots(figsize=(25,8))

    durations.plot(ax = ax1)

    ax1.set_xlabel('Trial Number')
    ax1.set_ylabel(label_y)
    ax1.set_xticks(np.arange(0, 21, step=1))

    plt.title(f'{title} over the Block')

    fig1.savefig(rf'{export_img}{filePrefix}_trialsBlock-plot.png')
    fig1.savefig(rf'{export_img}{filePrefix}_trialsBlock-plot.svg')

    plt.show()

    # Regression-style trend plot with jittered observations and mean estimator.
    sns.set_theme(font_scale = 1.5, palette=['#ea5b0c', '#1d71b8', '#3aaa35', '#e7267a'])
    sns.set_style("white")
    g = sns.lmplot(x='TrialId', y=columnName, hue='Condition', hue_order=condition_names, data = data_complete, height=8, aspect=2, x_estimator=np.mean, scatter=True, x_jitter=0.5, y_jitter=0.5)
    g.set(title=f'{title} over the Block')    
    g.set(xlabel='Trial Number')
    g.set(ylabel=label_y)
    g.set(xticks=np.arange(0, 21, step=1))

    for ax in plt.gcf().axes:
        add_margin(ax,x=0.05,y=0.01)

    g.savefig(rf'{export_img}{filePrefix}_trialsBlock-reg.png')
    g.savefig(rf'{export_img}{filePrefix}_trialsBlock-reg.svg')

    # for ax in plt.gcf().axes:
    #     l = ax.get_xlabel()
    #     ax.set_xlabel(l, fontsize=15)
    #     l = ax.get_ylabel()
    #     ax.set_ylabel(l, fontsize=15)

    plt.show()

    # Box plot: per-condition distribution summary.
    fig2, ax2 = plt.subplots(figsize=(9,9))

    condition_duration.boxplot(ax = ax2) 

    ax2.set_xlabel('Condition')
    ax2.set_ylabel(label_y)
    ax2.yaxis.grid(False)
    ax2.xaxis.grid(False)

    plt.title(f'Median {title} per Condition')

    fig2.savefig(rf'{export_img}{filePrefix}_trialsCondition-box.png')
    fig2.savefig(rf'{export_img}{filePrefix}_trialsCondition-box.svg')

    plt.show()

    # Bar plot: mean per block and condition.
    fig3, ax3 = plt.subplots(figsize=(25,8))

    condition_duration_block.plot(ax = ax3, kind="bar")

    ax3.set_xlabel('Block')
    ax3.set_ylabel(label_y)

    plt.title(f'Median Trial {title} per Block and Condition')

    fig3.savefig(rf'{export_img}{filePrefix}_trialsBlockCondition-bar.png')
    fig3.savefig(rf'{export_img}{filePrefix}_trialsBlockCondition-bar.svg')

    plt.show()

    # Bar plot: same data, transposed orientation (condition x block).
    fig4, ax4 = plt.subplots(figsize=(25,8))

    condition_block_duration.plot(ax = ax4, kind="bar")

    ax4.set_xlabel('Condition')
    ax4.set_ylabel(label_y)

    plt.title(f'Median Trial {title} per Condition and Block')

    fig4.savefig(rf'{export_img}{filePrefix}_trialsConditionBlock-bar.png')
    fig4.savefig(rf'{export_img}{filePrefix}_trialsConditionBlock-bar.svg')

    plt.show()


## Generic Boxplot Statistics Routine
This cell defines a reusable grouped-statistics and boxplot function with optional export and extended metrics.


In [ ]:
def generateBoxPlotStats(data, groupCol, indexColumns, statsColumn, xLabel, yLabel, title, filename, save= False, show = False, reindexColumns = False, outliers = False, ylim = [], extendedStats = True):
    # Build grouped descriptive statistics for the selected metric.
    grouped = data.groupby(groupCol)

    desc = grouped[statsColumn].describe().transpose()

    if reindexColumns:
        desc = desc.reindex(columns=condition_names)   

    display(desc)

    # Pivot raw values into one column per group for boxplot rendering.
    desc_pivot = data.pivot_table(columns=groupCol, index=indexColumns, values=statsColumn)

    if reindexColumns:
        desc_pivot = desc_pivot.reindex(columns=condition_names)    

    # Optionally compute extended summary stats and export them.
    if extendedStats == True:
        desc2 = desc_pivot.describe(percentiles=percentiles)
        stats = pd.DataFrame(desc2).T
        stats_total = pd.DataFrame(data[statsColumn].describe()).T

        computeCI(stats)
        computeCI(stats_total)

        computeWhiskers(desc_pivot, stats, condition_names)

        stats_complete = pd.concat([stats, stats_total], ignore_index=True)

        display(stats_complete.T)

        stats_complete.to_csv(rf'{export_data}{filename}_stats.csv', sep=";", index=True)

    # Draw the distribution chart with optional outliers and y-limits.
    fig1, ax1 = plt.subplots(figsize=(8,8))

    sns.boxplot(data = desc_pivot, ax = ax1, showfliers=outliers)

    ax1.set_xlabel(xLabel)
    ax1.set_ylabel(yLabel)
    ax1.yaxis.grid(False)
    ax1.xaxis.grid(False)
    if len(ylim) == 2:
        ax1.set_ylim(ylim[0], ylim[1])

    plt.title(title)

    if save:  
        fig1.savefig(rf'{export_img}{filename}-box.png')
        fig1.savefig(rf'{export_img}{filename}-box.svg')
    
    if show: 
        plt.show()


## Peak Z-Difference Computation
This cell computes within-trial z-differences for ordered peaks while resetting differences at trial boundaries.


In [2]:
def computeDifferenceZ(peaks):
    # Ensure temporal order before computing sequential differences.
    peaks = peaks.sort_values(['Date']).reset_index()
    peaks['Diff_Z'] = peaks['Peak_Z'].diff()
    peaks['Diff_Trial'] = peaks['Trial'].diff()

    # Mark the first row as a trial boundary.
    peaks.loc[0, 'Diff_Trial'] = 1

    # Reset z-difference at trial boundaries to avoid cross-trial subtraction.
    peaks.loc[peaks['Diff_Trial'] != 0, 'Diff_Z'] = 0

    return peaks
